#Benchmark Protocol
##Hackseq 2016 - Group 4

(2) Create element specific bed files for CDS, UTR, exon (other), genes, transcripts, and promoters (+/- 2Kb around TSS)

In [11]:
%%bash

BASE_PATH=/hackathon/Hackathon_Project_4
cd $BASE_PATH/GENOME_ANNOTATIONS/

awk -F "\t" '{if ($3 == "CDS") print $0}' gencode.v19.annotation.chr22.gtf | \
awk '{print $1 "\t" $4 "\t" $5 "\t" $28"-"$18 "\t" "." "\t" $7}' | \
awk '{gsub(/;/,"");print}' | awk '{gsub(/"/,"");print}' > CDSs.chr22.bed

awk -F "\t" '{if ($3 == "UTR") print $0}' gencode.v19.annotation.chr22.gtf | \
awk '{print $1 "\t" $4 "\t" $5 "\t" $12"-"$18 "\t" "." "\t" $7}' | \
awk '{gsub(/;/,"");print}' | awk '{gsub(/"/,"");print}' > UTRs.chr22.bed

awk -F "\t" '{if ($3 == "exon") print $0}' gencode.v19.annotation.chr22.gtf | \
awk '{print $1 "\t" $4 "\t" $5 "\t" $28"-"$18 "\t" "." "\t" $7}' | \
awk '{gsub(/;/,"");print}' | awk '{gsub(/"/,"");print}' > Exons.chr22.bed

awk -F "\t" '{if ($3 == "gene") print $0}' gencode.v19.annotation.chr22.gtf | \
awk '{print $1 "\t" $4 "\t" $5 "\t" $10"-"$18 "\t" "." "\t" $7}' | \
awk '{gsub(/;/,"");print}' | awk '{gsub(/"/,"");print}' > Genes.chr22.bed

awk '{if ($3 == "transcript") print $1 "\t" $4 "\t" $5 "\t" $12 "\t" "." "\t" $7 \
"\t" $10 "\t" $18 "\t" $14}' gencode.v19.annotation.chr22.gtf | awk '{gsub(/;/,"");print}' | \
awk '{gsub(/"/,"");print}' > Transcripts.chr22.bed

awk '{if ($6 == "+") print $1 "\t" $2-2000 "\t" $2+2000 "\t" $4 "\t" $5 "\t" $6 \
"\t" $7 "\t" $8 "\t" $9 ; else print $1 "\t" $3-2000 "\t" $3+2000 "\t" $4 "\t" $5 "\t" \
$6 "\t" $7 "\t" $8 "\t" $9}' Transcripts.chr22.bed > Promoters.chr22.bed 


(3) Intersect variants with genetic context bed files 

In [10]:
%%bash

BASE_PATH=/hackathon/Hackathon_Project_4
cd $BASE_PATH/GIAB_DATA

$BASE_PATH/SOFTWARE/Benchmark-Tools/Determine-Variant-Context.sh \
GIAB-V3.3-chr22.bed GIAB-V3.3-chr22.Annotated.bed

Processing CDSs...
Processing UTRs...
Processing Other Exons...
Processing Introns...
Processing Promoters...


(4) Generate small test benchmark set for testing pipelines by intersecting variants with CTCF peaks

In [20]:
%%bash
source ~/.bashrc
BASE_PATH=/hackathon/Hackathon_Project_4
cd $BASE_PATH/GIAB_DATA

bedtools intersect -u -b GIAB-V3.3-chr22.bed -a $BASE_PATH/ENCODE_DATA_GM12878/PEAKS/ENCFF002CDP.bed | \
sort -k7,7rg | head -50 > peaks.bed

echo -e "chrom\tposition\tposition\tsnp_id\tgenotype\tmajor_allele\tminor_allele\tdepth\tgenotype_quality" \
> $BASE_PATH/BENCHMARK/Benchmark-Test-V1.txt

bedtools intersect -u -a GIAB-V3.3-chr22.bed -b peaks.bed | \
sort -k9,9rg | head -25 >> $BASE_PATH/BENCHMARK/Benchmark-Test-V1.txt

rm peaks.bed
head $BASE_PATH/BENCHMARK/Benchmark-Test-V1.txt | column -t

chrom  position  position  snp_id      genotype  major_allele  minor_allele  depth  genotype_quality
chr22  48888255  48888255  rs1034435   G/A       G             A             711    1439
chr22  33445237  33445237  rs73162102  A/C       A             C             605    1341
chr22  30787190  30787190  rs61583714  T/C       T             C             593    1279
chr22  39739628  39739628  rs137685    C/T       C             T             709    1117
chr22  33397045  33397045  rs73162048  T/C       T             C             627    1101
chr22  33396904  33396904  rs73162047  C/T       C             T             573    1098
chr22  36618077  36618077  rs68166856  A/C       A             C             690    1088
chr22  40329148  40329148  rs137984    G/GT      G             GT            638    1086
chr22  50648481  50648481  rs73187260  C/G       C             G             503    1051


sort: write failed: standard output: Broken pipe
sort: write error


(5) Generate benchmark dataset for any Bam File